# Sample Codes

In [ ]:
workers()


In [3]:
procs()


8-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7
 8

In [2]:
addprocs(7)

7-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8

In [4]:
r = remotecall(3, rand, 2, 2)

RemoteRef{Channel{Any}}(3,1,8)

In [5]:
b=fetch(r)

2x2 Array{Float64,2}:
 0.872384  0.590679
 0.211705  0.717656

In [6]:
s = @spawnat 2 rand(2,2)

RemoteRef{Channel{Any}}(2,1,10)

In [7]:
fetch(s)

2x2 Array{Float64,2}:
 0.815234  0.440878
 0.486163  0.305873

In [8]:
remotecall_fetch(2, rand, 2,2)

2x2 Array{Float64,2}:
 0.665293  0.0699237
 0.680652  0.877099 

In [9]:
remotecall_fetch(2, getindex,r,1,1)

0.8723836367482065

In [14]:
r = @spawn rand(2,2)

RemoteRef{Channel{Any}}(6,1,18)

In [15]:
fetch(r)

2x2 Array{Float64,2}:
 0.579     0.56269 
 0.166937  0.998792

In [ ]:
s = @spawn 1+fetch(r)

In [ ]:
fetch(s)

In [16]:
a=RemoteRef[]
for i in 1:10
    push!(a,@spawn sin(i))
end

In [17]:
a

10-element Array{RemoteRef{RemoteStore},1}:
 RemoteRef{Channel{Any}}(7,1,20)
 RemoteRef{Channel{Any}}(8,1,21)
 RemoteRef{Channel{Any}}(2,1,22)
 RemoteRef{Channel{Any}}(3,1,23)
 RemoteRef{Channel{Any}}(4,1,24)
 RemoteRef{Channel{Any}}(5,1,25)
 RemoteRef{Channel{Any}}(6,1,26)
 RemoteRef{Channel{Any}}(7,1,27)
 RemoteRef{Channel{Any}}(8,1,28)
 RemoteRef{Channel{Any}}(2,1,29)

In [24]:
@time sum(map(fetch,a))

1.4111883712180104

0.002662 seconds (1.05 k allocations: 61.563 KB)


In [21]:
@time @parallel (+) for i in a
  fetch(i)
end

1.4111883712180104

0.019793 seconds (10.88 k allocations: 840.851 KB)


In [ ]:
@time sum(map((x)->fetch(x),a))

In [ ]:
@everywhere function producer()
  for i in 1:3
    produce(i)
  end
end
task=Task(producer)
consume(task)

In [ ]:
consume(task)

In [ ]:
n=10;
@everywhere function ssin() 
    for i in 1:n
        produce(sin(i))
    end
end
p=Task(ssin)
@time reduce(+,map(x->consume(p),1:n))

In [ ]:
s=0
@time for i in 1:10000000
  s+=sin(i)
end
s

In [ ]:
@time @parallel (+) for i=1:10000000
  sin(i)
end

In [ ]:
@time reduce(+,(map(i->sin(i),1:10000000)))

In [ ]:
@time reduce(+,@parallel [sin(i) for i in 1:10000000])

In [ ]:
@time sum( map(1:10000000) do x
         sin(x);
     end
)

# Data Movement

# Parallel Maps/Loops

In [ ]:
@everywhere function count_heads(n)
    c::Int = 0
    for i=1:n
        c += randbool()
    end
    c
end


In [ ]:
@time count_heads(200000000)/200000000

In [ ]:
#require("count_heads")

@time begin
a = @spawn count_heads(100000000)
b = @spawn count_heads(100000000)
(fetch(a)+fetch(b))/200000000
end

In [ ]:
@time begin
nheads = @parallel (+) for i=1:200000000
    int(randbool())
end
nheads/200000000
end

In [ ]:
@time begin
    a=0
    for i=1:200000000
      a+=int(randbool())
    end
    a/200000000
end

In [ ]:
@time sum(map(x->int(randbool()),1:200000000))/200000000

In [ ]:
b = zeros(100000);

c=@parallel (hcat) for i=1:10
    i
end


In [ ]:
c  // assignment is local to the process in @parallel for

In [ ]:
print(c);